In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import argparse
import random
from model import KGCN
from data_loader import DataLoader
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [3]:
parser = argparse.ArgumentParser()

parser.add_argument('--dataset', type=str, default='movie', help='which dataset to use')
parser.add_argument('--aggregator', type=str, default='sum', help='which aggregator to use')
parser.add_argument('--n_epochs', type=int, default=10, help='the number of epochs')
parser.add_argument('--neighbor_sample_size', type=int, default=4, help='the number of neighbors to be sampled')
parser.add_argument('--dim', type=int, default=32, help='dimension of user and entity embeddings')
parser.add_argument('--n_iter', type=int, default=2, help='number of iterations when computing entity representation')
parser.add_argument('--batch_size', type=int, default=65536, help='batch size')
parser.add_argument('--l2_weight', type=float, default=1e-7, help='weight of l2 regularization')
parser.add_argument('--lr', type=float, default=2e-2, help='learning rate')
parser.add_argument('--ratio', type=float, default=1, help='size of training dataset')

args = parser.parse_args(['--lr', '2e-2'])

In [7]:
data_loader = DataLoader()
kg = data_loader.load_kg()
df_dataset = data_loader.load_data()
df_dataset.head()

Construct knowledge graph ...Done
Build dataset dataframe ...Done


,userId,movieId,label
0,0,2,0
1,0,29,0
2,0,32,0
3,0,50,0
4,0,112,0


In [8]:
max(df_dataset['movieId'])

106033

In [9]:
class KGCNDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        user_id = np.array(self.df.iloc[idx]['userId'])
        movie_id = np.array(self.df.iloc[idx]['movieId'])
        label = np.array(self.df.iloc[idx]['label'], dtype=np.float32)
        return user_id, movie_id, label

In [10]:
x_train, x_test, y_train, y_test = train_test_split(df_dataset, df_dataset['label'], test_size=0.2)
train_dataset = KGCNDataset(x_train)
test_dataset = KGCNDataset(x_test)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2)

In [11]:
from model import KGCN
num_user, num_entity, num_relation = data_loader.get_num()
user_encoder, entity_encoder, relation_encoder = data_loader.get_encoders()
net = KGCN(num_user, num_entity, num_relation, kg, args)
criterion = torch.nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [116]:
running_loss = 0.0

for i, (user_id, movie_id, label) in enumerate(train_loader):
    outputs = net(user_id, movie_id)
    loss = criterion(outputs, label)
    loss.backward()
    optimizer.zero_grad()
    optimizer.step()

    running_loss += loss.item()
    if i % 10 == 9:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
            (1, i + 1, running_loss / 10))
        running_loss = 0.0

[1,    10] loss: 1.121
[1,    20] loss: 1.015
[1,    30] loss: 0.590


IndexError: index 113572 is out of bounds for axis 0 with size 106034